In [1]:
import airbnb as ab
import pandas as pd
import locale as lo

In [2]:
apt_id = 36902451

In [3]:
def get_reviews(listing_id, locale='fr', page_size=50):

    api = ab.api.Api(randomize=True)
    api._session.headers['x-airbnb-locale'] = locale
    api._session.headers['accept-language'] = locale
    lo.setlocale(lo.LC_ALL, locale)
    
    reviews = []
    offset = 0
    reviews_count = 0
    has_more_reviews = True
    reviews_count = -1

    while has_more_reviews:
        page_reviews = api.get_reviews(
            listing_id=apt_id, 
            offset=offset, 
            limit=page_size)

        for review in page_reviews['reviews']:
            reviews.append(review)

        reviews_count = page_reviews['metadata']['reviews_count']

        if len(page_reviews['reviews']) < page_size:
            has_more_reviews = False
        else:
            offset += page_size

    print('len(reviews): {0}'.format(len(reviews)))
    print('reviews_count: {0}'.format(reviews_count))
    
    return reviews

In [5]:
apt_reviews = get_reviews(listing_id=apt_id)
apt_reviews

{'accept': 'application/json', 'accept-encoding': 'br, gzip, deflate', 'content-type': 'application/json', 'x-airbnb-api-key': '915pw2pnf4h1aiguhph5gc5b2', 'user-agent': 'Airbnb/16.25 iPhone/9.3.4 Type/Phone', 'x-airbnb-screensize': 'w=375.00;h=812.00', 'x-airbnb-carrier-name': 'T-Mobile', 'x-airbnb-network-type': 'wifi', 'x-airbnb-currency': 'USD', 'x-airbnb-locale': 'fr', 'x-airbnb-carrier-country': 'us', 'accept-language': 'fr', 'airbnb-device-id': 'c86c7dee2abe97cf7d4c605de79ab1f566fadae0', 'x-airbnb-advertising-id': '10DF4335-CCB3-4EF4-88CF-AABDFCCE459B'}
len(reviews): 31
reviews_count: 31


[{'author': {'first_name': 'Camille',
   'has_profile_pic': True,
   'id': 8054257,
   'picture_url': 'https://a0.muscache.com/im/pictures/user/da31231b-666d-4167-a02e-9e619c0dc22c.jpg?aki_policy=profile_x_medium',
   'smart_name': 'Camille',
   'thumbnail_url': 'https://a0.muscache.com/im/pictures/user/da31231b-666d-4167-a02e-9e619c0dc22c.jpg?aki_policy=profile_small'},
  'author_id': 8054257,
  'can_be_edited': False,
  'comments': 'Je recommande vivement cet appartement. Très bien situé dans Chamonix, un appartement fonctionnel et propre, avec une très belle vue sur le Mont-Blanc. Patricia et Thierry sont très accueillants, arrangeants et disponibles. J’y retournerai sans hésitation!',
  'created_at': '2021-04-03T11:33:57Z',
  'id': 744854651,
  'id_str': '744854651',
  'listing_id': 36902451,
  'recipient_id': 251507728,
  'collection_tag': None,
  'listing': {'id': 36902451,
   'listing_id_str': '36902451',
   'name': 'Duplex spacieux et lumineux au centre de Chamonix'},
  'rating

In [6]:
def get_ratings(listing_id, locale='fr'):
    
    api = ab.api.Api(randomize=True)
    api._session.headers['x-airbnb-locale'] = locale
    api._session.headers['accept-language'] = locale
    lo.setlocale(lo.LC_ALL, locale)
    
    url_base = ab.api.API_URL
    
    r = api._session.get(
    '{0}/pdp_listing_details/{1}'.format(
        url_base,
        apt_id), 
    params={ '_format': 'for_rooms_show'})
    
    r.raise_for_status()
    listing_details = r.json()
    
    overall_rating_text = listing_details['pdp_listing_detail']['reviews_module']['localized_overall_rating']
    
    review_details = listing_details['pdp_listing_detail']['review_details_interface']

    overall_rating = lo.atof(overall_rating_text)
    review_count = review_details['review_count']
    review_summary = review_details['review_summary']
    
    for summary in review_summary:
        summary['rating'] = lo.atof(summary['localized_rating'])
        
    return {
        'overall_rating': overall_rating, 
        'review_count': review_count, 
        'categories': review_summary
    }

In [7]:
apt_ratings = get_ratings(listing_id=apt_id)
apt_ratings

{'overall_rating': 4.97,
 'review_count': 31,
 'categories': [{'category': 'accuracy',
   'value': 10,
   'label': 'Précision',
   'localized_rating': '5,0',
   'percentage': 1.0,
   'rating': 5.0},
  {'category': 'communication',
   'value': 10,
   'label': 'Communication',
   'localized_rating': '5,0',
   'percentage': 1.0,
   'rating': 5.0},
  {'category': 'cleanliness',
   'value': 10,
   'label': 'Propreté',
   'localized_rating': '5,0',
   'percentage': 0.994,
   'rating': 5.0},
  {'category': 'location',
   'value': 10,
   'label': 'Emplacement',
   'localized_rating': '5,0',
   'percentage': 1.0,
   'rating': 5.0},
  {'category': 'checkin',
   'value': 10,
   'label': 'Arrivée',
   'localized_rating': '5,0',
   'percentage': 1.0,
   'rating': 5.0},
  {'category': 'value',
   'value': 10,
   'label': 'Qualité-prix',
   'localized_rating': '5,0',
   'percentage': 0.994,
   'rating': 5.0}]}

In [21]:
pd.DataFrame.from_records(apt_reviews)['rating'].value_counts()

5    30
4     1
Name: rating, dtype: int64

In [84]:
# 'p3_display_review_summary', 'review_details_interface', 'sorted_reviews', 'visible_review_count'
listing_details['pdp_listing_detail']['review_details_interface']

{'display_review_summary': True,
 'host_other_property_review_count': 0,
 'review_count': 224,
 'review_score': 97,
 'review_summary': [{'category': 'accuracy',
   'value': 10,
   'label': 'Précision',
   'localized_rating': '4,9',
   'percentage': 0.984},
  {'category': 'communication',
   'value': 10,
   'label': 'Communication',
   'localized_rating': '4,9',
   'percentage': 0.984},
  {'category': 'cleanliness',
   'value': 10,
   'label': 'Propreté',
   'localized_rating': '4,8',
   'percentage': 0.97},
  {'category': 'location',
   'value': 10,
   'label': 'Emplacement',
   'localized_rating': '4,8',
   'percentage': 0.968},
  {'category': 'checkin',
   'value': 10,
   'label': 'Arrivée',
   'localized_rating': '5,0',
   'percentage': 0.99},
  {'category': 'value',
   'value': 10,
   'label': 'Qualité-prix',
   'localized_rating': '4,8',
   'percentage': 0.966}],
 'review_sorting_options': None,
 'n_reviews_per_page': 7,
 'review_tag_summary': None}

In [76]:
reviews = get_all_reviews(
    listing_id=apt_id)

{'accept': 'application/json', 'accept-encoding': 'br, gzip, deflate', 'content-type': 'application/json', 'x-airbnb-api-key': '915pw2pnf4h1aiguhph5gc5b2', 'user-agent': 'Airbnb/16.08 iPhone/10.2.1 Type/Phone', 'x-airbnb-screensize': 'w=375.00;h=812.00', 'x-airbnb-carrier-name': 'T-Mobile', 'x-airbnb-network-type': 'wifi', 'x-airbnb-currency': 'EUR', 'x-airbnb-locale': 'fr', 'x-airbnb-carrier-country': 'us', 'accept-language': 'fr-fr', 'airbnb-device-id': 'eba5c76e4b68dc8f61c9e38878f1747cc7730143', 'x-airbnb-advertising-id': '47A5CEE6-93D4-4D0D-87B8-F2F6BD861AE0'}
{'accept': 'application/json', 'accept-encoding': 'br, gzip, deflate', 'content-type': 'application/json', 'x-airbnb-api-key': '915pw2pnf4h1aiguhph5gc5b2', 'user-agent': 'Airbnb/17.08 iPhone/10.0.2 Type/Phone', 'x-airbnb-screensize': 'w=375.00;h=812.00', 'x-airbnb-carrier-name': 'T-Mobile', 'x-airbnb-network-type': 'wifi', 'x-airbnb-currency': 'EUR', 'x-airbnb-locale': 'fr', 'x-airbnb-carrier-country': 'us', 'accept-language'

In [54]:
reviews.keys()

dict_keys(['reviews', 'metadata'])

In [55]:
reviews['metadata']['reviews_count']

{'reviews_count': 224, 'should_show_review_translations': False}

In [56]:
reviews['reviews'][0]['comments']

'Super séjour, Isidre est très accueillant et chaleureux donnant pleins de conseils pour faciliter votre séjour à Barcelone.\nJe recommande vivement cet appartement'

In [57]:
reviews['reviews'][0]

{'author': {'first_name': 'Sylvain',
  'has_profile_pic': True,
  'id': 66291166,
  'picture_url': 'https://a0.muscache.com/im/pictures/user/6a177768-c026-448f-bcfd-30c6bfa5d40b.jpg?aki_policy=profile_x_medium',
  'smart_name': 'Sylvain',
  'thumbnail_url': 'https://a0.muscache.com/im/pictures/user/6a177768-c026-448f-bcfd-30c6bfa5d40b.jpg?aki_policy=profile_small'},
 'author_id': 66291166,
 'can_be_edited': False,
 'comments': 'Super séjour, Isidre est très accueillant et chaleureux donnant pleins de conseils pour faciliter votre séjour à Barcelone.\nJe recommande vivement cet appartement',
 'created_at': '2020-02-15T15:40:34Z',
 'id': 605803192,
 'id_str': '605803192',
 'listing_id': 6709228,
 'recipient_id': 35124028,
 'collection_tag': None,
 'listing': {'id': 6709228,
  'listing_id_str': '6709228',
  'name': 'Joli appartement très bien desservi'},
 'rating': 4,
 'recipient': {'first_name': 'Isidre',
  'has_profile_pic': True,
  'id': 35124028,
  'picture_url': 'https://a0.muscache.

In [58]:
# api.get_listing_details(listing_id=apt_id)

In [59]:
r = api._session.get(
    '{0}/pdp_listing_details/{1}'.format(
        url_base,
        apt_id), 
    params={ '_format': 'for_rooms_show'})
r.raise_for_status()

In [60]:
listing_details = r.json()

In [78]:
listing_details['pdp_listing_detail'].keys()

dict_keys(['listing_amenities', 'root_amenity_sections', 'see_all_amenity_sections', 'additional_house_rules', 'bathroom_label', 'bed_label', 'bedroom_label', 'guest_label', 'highlights', 'id', 'listing_expectations', 'listing_rooms', 'name', 'p3_subject', 'p3_summary_address', 'p3_summary_title', 'person_capacity', 'photos', 'primary_host', 'review_highlight', 'room_and_property_type', 'room_type_category', 'sectioned_description', 'star_rating', 'tier_id', 'user', 'book_it_url', 'calendar_last_updated_at', 'guest_controls', 'localized_minimum_nights_description', 'min_nights', 'native_currency', 'collection_kicker', 'show_policy_details', 'has_we_work_location', 'is_business_travel_ready', 'additional_hosts', 'applicable_disaster', 'hometour_rooms', 'hometour_sections', 'alternate_sectioned_description_for_p3', 'initial_description_author_type', 'localized_check_in_time_window', 'localized_check_out_time', 'localized_city', 'localized_listing_expectations', 'localized_room_type', 'ci

In [61]:
overall_rating = float(listing_details['pdp_listing_detail']['reviews_module']['localized_overall_rating'].replace(',', '.'))
overall_rating

4.83